In [32]:
import pyautogui
import time
import sys
from pathlib import Path
from typing import Optional, Tuple
import pandas as pd
import numpy as np
from PIL import Image
from bs4 import BeautifulSoup

In [33]:
class ImageFinder:
    def __init__(self,verbous=False):
        # Configure PyAutoGUI
        pyautogui.PAUSE = 1
        pyautogui.FAILSAFE = True
        
        # Screen size
        self.verbous = verbous
        self.screen_width, self.screen_height = pyautogui.size()
        if self.verbous: print(f"Screen size: {self.screen_width}x{self.screen_height}")

    def get_image_size(self,image_path):
        with Image.open(image_path) as img:
            width, height = img.size
            if self.verbous: print(f"Image size: {width}x{height} pixels")
            return width, height

    def validate_image_path(self, image_path: str) -> bool:
        """Validate if image file exists"""
        path = Path(image_path)
        if not path.exists():
            if self.verbous:print(f"Error: Image file not found: {image_path}")
            return False
        if not path.suffix.lower() in ['.png', '.jpg', '.jpeg']:
            if self.verbous:print(f"Error: Unsupported image format: {path.suffix}")
            return False
        return True

    def find_image(self, 
                  image_path: str, 
                  confidence: float = 0.9, 
                  region: Optional[Tuple[int, int, int, int]] = None,
                  grayscale: bool = False) -> Optional[Tuple[int, int]]:
        """
        Find image on screen with advanced options
        
        Args:
            image_path: Path to image file
            confidence: Match confidence (0-1)
            region: Search region (left, top, width, height)
            grayscale: Use grayscale matching
        """
        if not self.validate_image_path(image_path):
            if self.verbous:print(f"Not a valid image path to search for.")
            return None

        try:
            if self.verbous:print(f"Searching for image: {image_path}")
            location = pyautogui.locateOnScreen(
                image_path,
                confidence=confidence,
                region=region,
                grayscale=grayscale
            )
            
            if location:
                x, y = pyautogui.center(location)
                if self.verbous:print(f"Image found at: x={x}, y={y}")
                return (x, y)
            else:
                if self.verbous:print("Image not found")
                return None

        except Exception as e:
            if self.verbous:print(f"Error during image search: {str(e)}")
            return None

    def wait_for_image(self, 
                      image_path: str, 
                      timeout: int = 10, 
                      confidence: float = 0.8) -> Optional[Tuple[int, int]]:
        """Wait for image to appear within timeout period"""
        if self.verbous:print(f"Waiting for image (timeout: {timeout}s)...")
        start_time = time.time()
        
        while time.time() - start_time < timeout:
            position = self.find_image(image_path, confidence)
            if position:
                return position
            time.sleep(1)
            
        if self.verbous:print(f"Image not found after {timeout} seconds")
        return None

    def click_image(self, 
                   image_path: str, 
                   confidence: float = 0.8,
                   clicks: int = 1) -> bool:
        """Find and click on image"""
        position = self.find_image(image_path, confidence)
        if position:
            x, y = position
            pyautogui.click(x, y, clicks=clicks)
            if self.verbous:print(f"Clicked at position: x={x}, y={y}")
            return True
        return False


In [34]:
# pip3 install seleniumbase
from seleniumbase import Driver
import requests
from time import sleep

# initialize the driver in GUI mode with UC enabled
driver = Driver(uc=True, headless=False)

# open URL with a 6-second reconnect time to bypass the initial JS challenge
driver.uc_open_with_reconnect(f"https://dexscreener.com/", reconnect_time=6)
sleep(1)

finder = ImageFinder(verbous=False)
cloudFareBtn = "./resources/1-cloudfare_button.PNG"

# Dataframe to save the data
df = pd.DataFrame(columns=["chain","pair_contract","base_token_contract","symbol","name","volume","liquidity","mcap","age"])
chainName = ""
pairContractAddress = ""
symbol = ""
name = ""
volume = ""
liquidity = ""
mcap = ""
age = ""
baseTokenAddress = ""

chain = "solana"

if finder.wait_for_image(cloudFareBtn):
    finder.click_image(cloudFareBtn)

    sleep(.2)

    for i in range(1,1000):
        print(f"Getting data for page {i}")
        
        #driver.get(f'https://dexscreener.com/page-{i}?chainIds=ethereum&maxAge=168&order=desc&rankBy=volume')
        driver.get(f"https://dexscreener.com/{chain}/page-{i}?maxAge=336&min24HVol=1000&minLiq=1000&order=desc&rankBy=volume")
        soup = BeautifulSoup(driver.page_source, "html.parser")
        elements = soup.find("main", )
        elements = elements.findChild("div", attrs={"class":"ds-dex-table"})
        elements = elements.findChildren("a")
        for e in elements:
            chainName = e["href"].split("/")[1]
            pairContractAddress = e["href"].split("/")[2]
            symbol = e.find("span", attrs={"class":"ds-dex-table-row-base-token-symbol"}).text
            name = e.find("span", attrs={"class":"ds-dex-table-row-base-token-name-text"}).text

            volume = e.find("div", attrs={"class":"ds-table-data-cell ds-dex-table-row-col-volume"}).text.replace("$","")
            if "K" in volume: volume = float(float(volume.replace("K",""))*1000 )
            elif "M" in volume: volume = float(float(volume.replace("M",""))*1000000 )
            elif "B" in volume: volume = float(float(volume.replace("B",""))*1000000000 )
            else: 
                try: volume = float(volume)
                except: pass

            liquidity = e.find("div", attrs={"class":"ds-dex-table-row-col-liquidity"}).text.replace("$","")
            if "K" in liquidity: liquidity = float(float(liquidity.replace("K",""))*1000 )
            elif "M" in liquidity: liquidity = float(float(liquidity.replace("M",""))*1000000 )
            elif "B" in liquidity: liquidity = float(float(liquidity.replace("B",""))*1000000000 )
            else: 
                try: liquidity = float(liquidity)
                except: pass

            mcap = e.find("div", attrs={"class":"ds-dex-table-row-col-market-cap"}).text.replace("$","")
            if "K" in mcap: mcap = float(float(mcap.replace("K",""))*1000 )
            elif "M" in mcap: mcap = float(float(mcap.replace("M",""))*1000000 )
            elif "B" in mcap: mcap = float(float(mcap.replace("B",""))*1000000000 )
            else: 
                try: mcap = float(mcap)
                except: pass

            # Parse age to seconds {Xm, Xh, Xd, Xw}
            age = e.find("div", attrs = {"class":"ds-dex-table-row-col-pair-age"}).text
            age_number = int(age[:-1])
            age_unit = age[-1]

            if age_unit == 'm':
                age = age_number * 60  # minutes to seconds
            elif age_unit == 'h':
                age = age_number * 3600  # hours to seconds
            elif age_unit == 'd':
                age = age_number * 86400  # days to seconds
            elif age_unit == 'w':
                age = age_number * 604800  # weeks to seconds
            else:
                age = -1
                print(f"Invalid age_unit: {age_unit}. Use 'm' for minutes, 'h' for hours, or 'd' for days")

            response = requests.get(
                f"https://api.dexscreener.com/latest/dex/pairs/{chainName}/{pairContractAddress}",
                headers={},
            )
            data = response.json()

            if data["pairs"] != None:
                # For handeling the case-sensitivity of solana blockchain addresses
                pairContractAddress = data["pairs"][0]["pairAddress"]
                baseTokenAddress = data["pairs"][0]["baseToken"]["address"]
                sleep(.21)
                """print(s)
                pprint(str(data["pairs"][0]["pairAddress"]))

                print("base token")
                pprint(str(data["pairs"][0]["baseToken"]["address"]))
                pprint(str(data["pairs"][0]["baseToken"]["name"]))

                print("quote token")
                pprint(str(data["pairs"][0]["quoteToken"]["address"]))
                pprint(str(data["pairs"][0]["quoteToken"]["name"]))"""
            else:
                # if request was rejected, set base token conteract as an empty string so that 
                # entire iteration isn't in vain
                baseTokenAddress = ""

            df = pd.concat([df, pd.DataFrame({
                "chain":chainName,
                "pair_contract":pairContractAddress,
                "base_token_coneract":baseTokenAddress,
                "symbol":symbol,
                "name":name,
                "volume":volume,
                "liquidity":liquidity,
                "mcap":mcap,
                "age":age
                }, index=[0])], ignore_index=True)
            
        if elements == None: driver.quit(); break
        if not elements: driver.quit(); break
else:
    raise Exception("Cloudfare button not found.")



Getting data for page 1


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_6088\2000703535.py:117: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame({


Getting data for page 2
Getting data for page 3
Getting data for page 4
Getting data for page 5
Getting data for page 6
Getting data for page 7
Getting data for page 8
Getting data for page 9
Getting data for page 10
Getting data for page 11
Getting data for page 12
Getting data for page 13
Getting data for page 14
Getting data for page 15
Getting data for page 16
Getting data for page 17
Getting data for page 18
Getting data for page 19
Getting data for page 20
Getting data for page 21
Getting data for page 22
Getting data for page 23
Getting data for page 24
Getting data for page 25
Getting data for page 26
Getting data for page 27
Getting data for page 28
Getting data for page 29
Getting data for page 30


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=132.0.6834.83)
Stacktrace:
	GetHandleVerifier [0x00007FF70B67FBE5+28741]
	(No symbol) [0x00007FF70B5E7890]
	(No symbol) [0x00007FF70B484FDA]
	(No symbol) [0x00007FF70B45ED45]
	(No symbol) [0x00007FF70B506EC7]
	(No symbol) [0x00007FF70B51F982]
	(No symbol) [0x00007FF70B4FFCB3]
	(No symbol) [0x00007FF70B4C9FB3]
	(No symbol) [0x00007FF70B4CB331]
	GetHandleVerifier [0x00007FF70B9BA73D+3414941]
	GetHandleVerifier [0x00007FF70B9CE64A+3496618]
	GetHandleVerifier [0x00007FF70B9C413D+3454365]
	GetHandleVerifier [0x00007FF70B74848B+850155]
	(No symbol) [0x00007FF70B5F37FF]
	(No symbol) [0x00007FF70B5EF0C4]
	(No symbol) [0x00007FF70B5EF25D]
	(No symbol) [0x00007FF70B5DE079]
	BaseThreadInitThunk [0x00007FFD34EF7E94+20]
	RtlUserThreadStart [0x00007FFD35547AD1+33]


In [ ]:
df.to_csv("tmp.csv")

In [ ]:
df.head()

,chain,pair_contract,symbol,name,volume,liquidity,mcap,age
0,solana,fpz8ddmguzdyx6d8fkggw4qvfzgsnygnhkniwqobbdcm,RETIREMENT,THE RETIREMENT COIN,2.800000e+09,12000.0,118700000.0,345600
1,solana,3bfv8sblvf4xuudtwdc7joglnadvbtasgzk86vvphsj4,FABS,Founders Abs,2.640000e+09,1300.0,16000.0,432000
2,solana,fwhnmyzgbvl6emsal1xwc4apadgmkct12gpuxafwvfjx,EVIVO,EVIVO Token,2.030000e+09,22000.0,1000000.0,432000
3,solana,bqexvmmrwwjfy57agxvor4fhwssfpnefuxcmft1nfw2x,NUKEDOG,NUKEDOG,1.780000e+09,1200000.0,348500000.0,345600
4,solana,g6xzu9m4yamow1ugfkcahuw5ufpurbk4kcewr9dp1kxj,LLM,Large Language Model,1.345000e+08,1500000.0,60700000.0,28800
